# **mohammad amin kiani 4003613052**

# load movie_reviews Corpus from NLTK:

In [26]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
import random

import string
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
import re
import math
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# پاکسازی کامل کش‌های خراب
nltk.download('popular')
nltk.download('all')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_

True

# All docs

In [2]:
# لیست مستندات و برچسب‌ها
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# درهم‌ریزی برای train/test
random.shuffle(documents)

# تعداد کل و توزیع برچسب‌ها
print(f"تعداد کل مستندات: {len(documents)}")
from collections import Counter
print("توزیع برچسب‌ها:", Counter([label for (text, label) in documents]))

تعداد کل مستندات: 2000
توزیع برچسب‌ها: Counter({'pos': 1000, 'neg': 1000})


# feature extraction

In [3]:
# استخراج واژگان مهم
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]  # 2000 کلمه برتر

# تابع استخراج ویژگی از متن
def document_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[f'contains({w})'] = (w in words)
    return features

# ایجاد مجموعه ویژگی‌ها
featuresets = [(document_features(d), c) for (d, c) in documents]


In [40]:
# stop_words = set(stopwords.words('english'))
# punct = set(string.punctuation)

# #   پیش‌پردازش :
# def clean_tokens(tokens):
#     return [
#         w.lower() for w in tokens
#         if w.lower() not in stop_words              # حذف stopword
#         and w not in punct                          # حذف علائم نگارشی
#         and len(w) > 1                              # حذف کلمات خیلی کوتاه
#         and w.isalpha()                             # فقط حروف الفبا
#         and not re.match(r'https?://\S+|\d+', w)    # حذف URL و عدد
#     ]

# # بارگذاری داده‌ها:
# documents = []
# for category in movie_reviews.categories():
#     for fileid in movie_reviews.fileids(category):
#         words = clean_tokens(movie_reviews.words(fileid))
#         documents.append((words, category))

# random.shuffle(documents)

#  توقف، علائم،  regex
stop_words = set(stopwords.words('english'))
punct = set(string.punctuation)

# تابع پیش‌پردازش
def clean_tokens(tokens):
    return [
        w.lower() for w in tokens
        if w.lower() not in stop_words
        and w not in punct
        and len(w) > 2
        and w.isalpha()
        and not re.match(r'https?://\S+|\d+', w)
    ]

# بارگذاری داده‌ها
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        words = clean_tokens(movie_reviews.words(fileid))
        documents.append((words, category))

random.shuffle(documents)


# lookout dataset

In [47]:
stop_words = set(stopwords.words('english'))
punct = set(string.punctuation)
custom_garbage = {'just', 'really', 'get', 'got', 'even', 'thing', 'things', 'also', 'though'}

# پیش‌پردازش کامل
def clean_tokens(tokens):
    return [
        w.lower() for w in tokens
        if w.lower() not in stop_words
        and w.lower() not in custom_garbage
        and w not in punct
        and len(w) > 2
        and w.isalpha()
        and not re.match(r'https?://\S+|\d+', w)
    ]

# بارگذاری داده‌ها
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        words = clean_tokens(movie_reviews.words(fileid))
        documents.append((words, category))

random.shuffle(documents)

# Create Model

آماده:

In [4]:
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy

# تقسیم آموزش/تست
train_set, test_set = featuresets[:1600], featuresets[1600:]

# آموزش مدل
classifier = NaiveBayesClassifier.train(train_set)

# دقت مدل
print("Accuracy:", accuracy(classifier, test_set))


Accuracy: 0.85


دستی:

In [49]:
# # تقسیم آموزش/تست
# split_idx = int(0.7 * len(documents))
# train_docs = documents[:split_idx]
# test_docs = documents[split_idx:]

# # مدل‌سازی
# class_word_counts = {'pos': Counter(), 'neg': Counter()}
# class_doc_counts = {'pos': 0, 'neg': 0}
# total_vocab = Counter()

# for words, label in train_docs:
#     class_doc_counts[label] += 1
#     class_word_counts[label].update(words)
#     total_vocab.update(words)

# # فیلتر کلمات نادر از واژگان
# vocab = {w for w, c in total_vocab.items() if c >= 3}
# V = len(vocab)

# # به‌روزرسانی مدل‌ها بعد از حذف کلمات نادر
# for cls in ['pos', 'neg']:
#     class_word_counts[cls] = Counter({w: c for w, c in class_word_counts[cls].items() if w in vocab})

# تقسیم به آموزش/تست
split_idx = int(0.7 * len(documents))
train_docs = documents[:split_idx]
test_docs = documents[split_idx:]

# ساخت واژگان و شمارش اولیه
class_word_counts = {'pos': Counter(), 'neg': Counter()}
class_doc_counts = {'pos': 0, 'neg': 0}
total_vocab = Counter()

for words, label in train_docs:
    class_doc_counts[label] += 1
    class_word_counts[label].update(words)
    total_vocab.update(words)

# فیلتر واژه‌های کم‌تکرار
vocab = {w for w, c in total_vocab.items() if c >= 3}
# محدود به 4000 واژه مهم
top_words = set([w for w, c in total_vocab.most_common(4000)])
vocab = vocab.intersection(top_words)
V = len(vocab)

# به‌روزرسانی مدل بر اساس واژگان بهبود یافته
for cls in ['pos', 'neg']:
    class_word_counts[cls] = Counter({w: c for w, c in class_word_counts[cls].items() if w in vocab})

print(V)
print("-----")
vocab

4000
-----


{'secretly',
 'strange',
 'officer',
 'mia',
 'great',
 'imaginative',
 'clever',
 'develops',
 'excitement',
 'hundred',
 'edition',
 'catches',
 'halloween',
 'lake',
 'accept',
 'unlikely',
 'else',
 'emily',
 'achieved',
 'stranger',
 'sentimental',
 'perfectly',
 'charismatic',
 'levels',
 'jawbreaker',
 'wahlberg',
 'creates',
 'portrayal',
 'patch',
 'letter',
 'ways',
 'nightmare',
 'thoughts',
 'robbery',
 'clooney',
 'fantasies',
 'soul',
 'imagery',
 'saving',
 'depp',
 'cameo',
 'wonderful',
 'fits',
 'admit',
 'lit',
 'suffer',
 'native',
 'battle',
 'casting',
 'first',
 'christopher',
 'alice',
 'sat',
 'searching',
 'jennifer',
 'eddie',
 'obsession',
 'remarkably',
 'cannot',
 'turn',
 'drop',
 'tree',
 'research',
 'tape',
 'planning',
 'ads',
 'battles',
 'evening',
 'lifetime',
 'edge',
 'todd',
 'blake',
 'bring',
 'expecting',
 'execution',
 'sorts',
 'tall',
 'bacon',
 'hannah',
 'friendly',
 'cop',
 'shall',
 'bottle',
 'patients',
 'reviews',
 'god',
 'predicta

In [52]:
# def predict_naive_bayes(text_tokens):
#     # پیش‌پردازش
#     tokens = clean_tokens(text_tokens)

#     scores = {}
#     total_docs = sum(class_doc_counts.values())

#     for cls in ['pos', 'neg']:
#         # log(P(class))
#         log_prob = math.log(class_doc_counts[cls] / total_docs)

#         # log(P(w | class)) مجموعی
#         for word in tokens:
#             word_count = class_word_counts[cls][word]
#             total_words = sum(class_word_counts[cls].values())
#             prob = (word_count + 1) / (total_words + V)  # Laplace smoothing
#             log_prob += math.log(prob)

#         scores[cls] = log_prob

#     return max(scores, key=scores.get)

def predict_naive_bayes(text_tokens):
    tokens = clean_tokens(text_tokens)
    scores = {}
    total_docs = sum(class_doc_counts.values())

    for cls in ['pos', 'neg']:
        log_prob = math.log(class_doc_counts[cls] / total_docs)
        if cls == 'pos':
            log_prob += 0.2  # تقویت prior کلاس مثبت

        total_words = sum(class_word_counts[cls].values())

        for word in tokens:
            if word in vocab:
                count = class_word_counts[cls][word]
                numerator = math.log(count + 1)
                denominator = math.log(total_words + V)
                log_prob += numerator - denominator

        scores[cls] = log_prob

    return max(scores, key=scores.get)


In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import math

y_true = []
y_pred = []

for text, label in test_docs:
    y_true.append(label)
    y_pred.append(predict_naive_bayes(text))

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, pos_label='pos'))
print("Recall:", recall_score(y_true, y_pred, pos_label='pos'))
print("F1 Score:", f1_score(y_true, y_pred, pos_label='pos'))


Accuracy: 0.8166666666666667
Precision: 0.8485915492957746
Recall: 0.7824675324675324
F1 Score: 0.8141891891891891


In [53]:
y_true, y_pred = [], []

for words, label in test_docs:
    y_true.append(label)
    y_pred.append(predict_naive_bayes(words))

print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

         neg     0.8249    0.8221    0.8235       298
         pos     0.8251    0.8278    0.8264       302

    accuracy                         0.8250       600
   macro avg     0.8250    0.8250    0.8250       600
weighted avg     0.8250    0.8250    0.8250       600



# Eval

In [5]:
from sklearn.metrics import classification_report

# پیش‌بینی و مقایسه برچسب‌ها
test_gold = [label for (features, label) in test_set]
test_pred = [classifier.classify(features) for (features, label) in test_set]

print(classification_report(test_gold, test_pred, digits=4))


              precision    recall  f1-score   support

         neg     0.8125    0.8895    0.8492       190
         pos     0.8906    0.8143    0.8507       210

    accuracy                         0.8500       400
   macro avg     0.8516    0.8519    0.8500       400
weighted avg     0.8535    0.8500    0.8500       400



In [46]:
classifier.show_most_informative_features(10)

Most Informative Features
        contains(seagal) = True              neg : pos    =     11.4 : 1.0
   contains(outstanding) = True              pos : neg    =      9.6 : 1.0
         contains(damon) = True              pos : neg    =      7.6 : 1.0
   contains(wonderfully) = True              pos : neg    =      5.9 : 1.0
    contains(ridiculous) = True              neg : pos    =      5.4 : 1.0
          contains(lame) = True              neg : pos    =      5.2 : 1.0
        contains(wasted) = True              neg : pos    =      5.1 : 1.0
         contains(awful) = True              neg : pos    =      5.0 : 1.0
         contains(waste) = True              neg : pos    =      4.6 : 1.0
         contains(flynt) = True              pos : neg    =      4.4 : 1.0
